<a href="https://colab.research.google.com/github/michalis0/DataMining_and_MachineLearning/blob/master/week3/Advanced_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Mining and Machine Learning - Week 3
# Advanced Pandas Operations

### Table of Contents
###### 1. Simple Aggregation
###### 2. Advanced aggregation topics and chaining
###### 3. Selecting using an aggregating dimension

Note: This notebook follows the sildes of the pdf file <a href='https://github.com/michalis0/DataMining_and_MachineLearning/blob/master/week3/Walk-Through-Tutorial/3.%20Pandas_Aggregate.pdf'>3. Pandas_Aggregate.pdf</a> provided in the github repository.

In [1]:
# Import required packages
import pandas as pd

In [2]:
# Read in data
myData = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/week3/data/transactions_students.csv", sep=",")
myData.head(3)

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID
0,149332,15.11.2005,1,199.95,107.0,127998739
1,172951,29.08.2008,1,199.95,108.0,128888288
2,120621,19.10.2007,1,99.95,49.0,125375247


In [3]:
# Get some information
myData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223191 entries, 0 to 223190
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Customer     223191 non-null  int64  
 1   TransDate    223191 non-null  object 
 2   Quantity     223191 non-null  int64  
 3   PurchAmount  223191 non-null  float64
 4   Cost         223191 non-null  float64
 5   TransID      223191 non-null  int64  
dtypes: float64(2), int64(3), object(1)
memory usage: 10.2+ MB


In [4]:
# Adjust the format of column "TransDate" to datetime.
myData["TransDate"]  = pd.to_datetime(myData["TransDate"], dayfirst=True)

# Alternative conversion: Convert date from string to date times
# import dateutil
# myData["TransDate"] = myData["TransDate"].apply(dateutil.parser.parse, dayfirst=True)

myData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223191 entries, 0 to 223190
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Customer     223191 non-null  int64         
 1   TransDate    223191 non-null  datetime64[ns]
 2   Quantity     223191 non-null  int64         
 3   PurchAmount  223191 non-null  float64       
 4   Cost         223191 non-null  float64       
 5   TransID      223191 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(3)
memory usage: 10.2 MB


In [5]:
pd.options.display.max_rows = 10 # just show 10 rows

### 1. Simple Aggregations
#### 1.1 Apply an aggregating function to a variable by an aggregating dimension 
###### Sum `PurchAmount` by `Customer`

In [6]:
myData.groupby("Customer", as_index=False)["PurchAmount"].sum()

,Customer,PurchAmount
0,100001,279.90
1,100002,499.95
2,100003,379.90
3,100004,499.95
4,100005,309.80
...,...,...
98775,199995,89.85
98776,199996,179.95
98777,199997,179.70
98778,199998,29.95


In [7]:
# alternative option: using agg
myData.groupby("Customer", as_index=False)["PurchAmount"].agg(["sum"]).rename(columns={"PurchAmount":"AggPurch"})

,sum
Customer,
100001,279.90
100002,499.95
100003,379.90
100004,499.95
100005,309.80
...,...
199995,89.85
199996,179.95
199997,179.70


#### 1.2 Apply multiple aggregation functions to a variable by an aggregating dimension
###### Sum and select max of `PurchAmount` by `Customer`

In [8]:
myData.groupby("Customer", as_index=False)["PurchAmount"].agg({"AggPurch": "sum", "Purch_max": "max"})

,Customer,AggPurch,Purch_max
0,100001,279.90,199.95
1,100002,499.95,499.95
2,100003,379.90,249.95
3,100004,499.95,499.95
4,100005,309.80,79.95
...,...,...,...
98775,199995,89.85,29.95
98776,199996,179.95,179.95
98777,199997,179.70,29.95
98778,199998,29.95,29.95


#### 1.3 Apply an aggregating function to the whole dataset
###### Sum all `PurchAmount`

In [9]:
myData["PurchAmount"].sum()

18784784.62

#### Slidenote: Create new columns with the transform()-function
Here we add a new column `AggPurch` which holds for every customer the total of her purchases.

In [10]:
myData["AggPurch"] = myData.groupby("Customer")["PurchAmount"].transform(sum)
myData.head()

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID,AggPurch
0,149332,2005-11-15,1,199.95,107.00,127998739,274.85
1,172951,2008-08-29,1,199.95,108.00,128888288,889.80
2,120621,2007-10-19,1,99.95,49.00,125375247,99.95
3,149236,2005-11-14,1,39.95,18.95,127996226,119.90
4,149236,2007-06-12,1,79.95,35.00,128670302,119.90


#### 1.4 Apply a/ multiple aggregating function(s) to multiple variables by an aggregating dimension
###### Apply various aggregation functions to `PurchAmount` and `Quantity` aggregated by `Customer`

In [11]:
myData.groupby("Customer", as_index=False).agg({"PurchAmount":["max","sum"], "Quantity":"sum"})

Customer PurchAmount         Quantity
                       max     sum      sum
0       100001      199.95  279.90        2
1       100002      499.95  499.95        1
2       100003      249.95  379.90        2
3       100004      499.95  499.95        1
4       100005       79.95  309.80        4
...        ...         ...     ...      ...
98775   199995       29.95   89.85        3
98776   199996      179.95  179.95        1
98777   199997       29.95  179.70        6
98778   199998       29.95   29.95        1
98779   199999      179.95  179.95        1

[98780 rows x 4 columns]

#### 1.5 Apply an aggregating function to multiple variables by multiple aggregating dimensions
###### Sum `PurchAmount` and sum `Quantity` aggregated by `Customer` and `TransDate`

In [12]:
myData.groupby(["Customer","TransDate"])[["PurchAmount","Quantity"]].sum()

PurchAmount  Quantity
Customer TransDate                        
100001   2011-06-25        79.95         1
         2011-08-24       199.95         1
100002   2004-12-29       499.95         1
100003   2012-01-23       379.90         2
100004   2012-08-05       499.95         1
...                          ...       ...
199995   2012-09-17        89.85         3
199996   2012-09-17       179.95         1
199997   2012-09-17       179.70         6
199998   2012-09-17        29.95         1
199999   2012-09-17       179.95         1

[135038 rows x 2 columns]

#### 1.6 Apply an aggregating function to a variable by an aggregating dimension to a selection of rows
###### Select rows 2 to 5 and sum `PurchAmount` by `Customer`

In [13]:
myData.iloc[1:6].groupby("Customer", as_index=False)["PurchAmount"].sum()

,Customer,PurchAmount
0,120621,99.95
1,140729,129.95
2,149236,119.90
3,172951,199.95


#### 1.7 Exercises
###### 1. Sum `PurchAmount` by `Customer` and `TransDate` (hint: group by both `Customer` and `TransDate`).
###### 2. Count the number of transactions by `Customer` (hint: use the size() function)

In [14]:
# 1. [your code here]

# 2. [your code here]

### 2. Advanced aggregation topics and chaining
#### 2.1 Aggregate a variable by a transformed aggregating dimension
###### Sum `PurchAmount` by month of each year

In [15]:
myData.groupby(myData["TransDate"].dt.to_period("M"))[["PurchAmount"]].sum()

,PurchAmount
TransDate,
2004-12,27623.90
2005-01,83363.73
2005-02,87341.59
2005-03,86803.31
2005-04,84293.01
...,...
2012-08,108462.20
2012-09,71429.25
2012-10,42588.75


#### Slidenote: Chaining saves memory and is faster
Sum `PurchAmount` by `Customer` and select Customers with aggregated sums greater than 100.

In [16]:
myData.groupby("Customer")[["PurchAmount"]].sum()[lambda x: x >= 100]

,PurchAmount
Customer,
100001,279.90
100002,499.95
100003,379.90
100004,499.95
100005,309.80
...,...
199995,NaN
199996,179.95
199997,179.70


In [17]:
# alternative
myData2 = myData.groupby("Customer", as_index=False)["PurchAmount"].agg({"AggPurch": "sum"})
myData2.loc[myData2["AggPurch"]>=100]

,Customer,AggPurch
0,100001,279.90
1,100002,499.95
2,100003,379.90
3,100004,499.95
4,100005,309.80
...,...,...
98769,199989,119.80
98771,199991,199.95
98776,199996,179.95
98777,199997,179.70


#### 2.2 Exercises
###### 1. Aggregate the purchase amount (sum) of all transactions per customer on a yearly basis for eyar 2007 and 2008.
###### 2. How many customers purchased more than 50$ in total between 2008 and 2009?

In [18]:
# Question 1.
# Part1 use the loc function and select the year 2007 OR 2008 using: dt.year=200X to select years 
# use the | to select 2007 or 2008
# part2: groupby([myData["TransDate"].dt.year, "Customer"])
# part 3: take the sum() on the PurchAmount
myData.loc[(myData["TransDate"].dt.year == ) | (myData["TransDate"].dt.year == )]\
.groupby([])\
["..."].sum()

SyntaxError: ignored

In [19]:
# Question 2. How many customers purchased for more than 50$ in total between 2008 and 2009

# part 1: select rows with year 2008 or 2009 using the loc function

# part 2: groupby("Customer")

# part 3: take the sum() on the PurchAmount

# take the count() on those aggregations that have sum more than 50$, hint [lambda x:x >50]

### 3. Selecting using an aggregating dimension
#### 3.1 Select the first 3 purchases of each customer

In [20]:
# sort by ascending order of "Customer"
myData.groupby("Customer").head(3).sort_values("Customer")

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID,AggPurch
198456,100001,2011-06-25,1,79.95,27.00,130025578,279.90
198455,100001,2011-08-24,1,199.95,90.00,123457297,279.90
44541,100002,2004-12-29,1,499.95,349.00,123490350,499.95
214846,100003,2012-01-23,1,129.95,66.92,123457694,379.90
214845,100003,2012-01-23,1,249.95,202.60,123457694,379.90
...,...,...,...,...,...,...,...
223182,199997,2012-09-17,1,29.95,13.80,132481149,179.70
223183,199997,2012-09-17,1,29.95,13.80,132481149,179.70
223184,199997,2012-09-17,1,29.95,13.80,132481149,179.70
223188,199998,2012-09-17,1,29.95,13.80,132481154,29.95


Note: use `tail(1)` insatead of `head(3)` to select the last purchase of each customer.

#### 3.2 Updating columns using an aggregating dimension
###### Add a column counting the quantities per customer

In [21]:
# save the total number of occurences per customer in the count column
myData["Count"] = myData.groupby("Customer")["Customer"].transform("size")
myData

#myData.groupby("Customer")["Customer"].transform("count")

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID,AggPurch,Count
0,149332,2005-11-15,1,199.95,107.00,127998739,274.85,3
1,172951,2008-08-29,1,199.95,108.00,128888288,889.80,4
2,120621,2007-10-19,1,99.95,49.00,125375247,99.95,1
3,149236,2005-11-14,1,39.95,18.95,127996226,119.90,2
4,149236,2007-06-12,1,79.95,35.00,128670302,119.90,2
...,...,...,...,...,...,...,...,...
223186,199997,2012-09-17,1,29.95,13.80,132481149,179.70,6
223187,199997,2012-09-17,1,29.95,13.80,132481149,179.70,6
223188,199998,2012-09-17,1,29.95,13.80,132481154,29.95,1
223189,199999,2012-09-17,1,179.95,109.99,132481165,179.95,1


###### Add a column counting the transactions per customer

In [22]:
# counts the total number of transactions per customer
myData["RelDate"] = myData.groupby("Customer").cumcount() + 1
myData

,Customer,TransDate,Quantity,PurchAmount,Cost,TransID,AggPurch,Count,RelDate
0,149332,2005-11-15,1,199.95,107.00,127998739,274.85,3,1
1,172951,2008-08-29,1,199.95,108.00,128888288,889.80,4,1
2,120621,2007-10-19,1,99.95,49.00,125375247,99.95,1,1
3,149236,2005-11-14,1,39.95,18.95,127996226,119.90,2,1
4,149236,2007-06-12,1,79.95,35.00,128670302,119.90,2,2
...,...,...,...,...,...,...,...,...,...
223186,199997,2012-09-17,1,29.95,13.80,132481149,179.70,6,5
223187,199997,2012-09-17,1,29.95,13.80,132481149,179.70,6,6
223188,199998,2012-09-17,1,29.95,13.80,132481154,29.95,1,1
223189,199999,2012-09-17,1,179.95,109.99,132481165,179.95,1,1


#### 3.3 Exercises
###### 1. Add a column to myData with the total number of purchases per customer (Hint: Use the transform("size") function).

In [ ]:
# your solution
myData["Count"]=...